In [193]:
import random
import collections
import operator
from functools import lru_cache


# Descripción de baraja inglesa

La baraja está dividida en cuatro palos (en inglés: suit), dos de color rojo y dos de color negro:

    ♠ → Espadas (conocidas como picas). NEGRO
    ♥ → Corazones (conocidos como copas). ROJO
    ♦ → Rombos (conocidos como diamantes, oros o cocos). ROJO
    ♣ → Tréboles (conocidos como flores o bastos). NEGRO

Cada palo está formado por 13 cartas, de las cuales 9 cartas son numerales y 4 literales. Se ordenan de menor a mayor "rango" de la siguiente forma: A ,2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K. Las cartas con letras, las figuras, se llaman jack, queen, king y ace. En español reciben nombres diversos, que se detallan más adelante.

Los dos comodines se denominan jokers (en singular joker), uno está a colores (amarillo, azul y rojo) y el otro está a blanco y negro. 

https://es.wikipedia.org/wiki/P%C3%B3quer

## Clasificación de las manos

    -Flor imperial
    -Escalera color
    -Poker
    -Fulls
    -Color - Cinco cartas del mismo palo, sin ser consecutivas. 
    -Escalera - Cinco cartas consecutivas sin importar el palo. B = {a_k, a_k+1, ..., a_r-1, a_r}
    -Trio
    -Doble pareja
    -Pareja
    -Carta alta

# Simulación de poker

In [18]:
PALOS = ['espada', 'corazon', 'rombo', 'trebol']
VALORES = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
#VALORES = [(1, 'A')[0], (2,'2')[0], (3,'3')[0], (4,'4')[0], (5,'5')[0], (6,'6')[0], (7,'7')[0], (8,'8')[0], (9,'9')[0], (10,'10')[0], (11,'J')[0], (12,'Q')[0], (13,'K')[0]]
#VALORES_DICT
VALORES_DICT = {1:'A', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'J', 12:'Q', 13:'K'}



In [4]:
list(enumerate(VALORES, start=1))

[(1, 'A'),
 (2, '2'),
 (3, '3'),
 (4, '4'),
 (5, '5'),
 (6, '6'),
 (7, '7'),
 (8, '8'),
 (9, '9'),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13)]

In [61]:
def generar_baraja():
    """
    Combinación de todos los valores con todos los palos
    """
    barajas = [ (palo, valor) for palo in PALOS for valor in list(enumerate(VALORES.copy(), start=1)) ]
    
    return barajas

In [6]:
def obtener_mano(barajas, tamano_mano):
    """
    Genera una mano de barajas
    """
    try:
        mano = random.sample(barajas, tamano_mano)
    except ValueError as e:
        print('Ese número de barajas no es valido.', e)

    return mano

# Generadores de manos

In [241]:
def escalera_real(mano):
    ESCALERA_REAL = [(palo, valor) for palo in PALOS for valor in VALORES if valor in enumerate(['A','J', 'Q', 'K', '10'], start=1)]
    MANOS = []
    for i in range(0,4*5,5):
        MANOS.append(ESCALERA_REAL[i:i+5])
    
    if mano in ESCALERA_REAL:
        return True
    else: 
        return False

In [217]:
def escalera_color(mano, tamano):
    palos_mano = { palo[0] for palo in mano }
    #print(f'Palos: {palos_mano}')
    if escalera(mano, tamano) and len(palos_mano) == 1:
        return True
    return False

In [216]:

def color(mano, tamano):
    palos_mano = { palo[0] for palo in mano }
    if len(palos_mano) == 1:
        return True
    return False

In [197]:

def escalera(mano, tamano):
    #Obtiene los valores númericos del 1 al 13 de cada carta de la baraja
    valores =  [x[0] for x in  list(enumerate(VALORES.copy(), start=1))]


    mano.sort(key = lambda x:x[1])
    #Va a tomar el primer elemento de la mano (es una tupla)
    indice = valores.index(mano[0][1][0])

    fragmento_valores = valores[indice:indice+tamano].copy()
    try:
        for a,b in zip(mano, fragmento_valores):
            #Va a tomar el valor número de la mano, el cual está determinado por un índice
            valor_mano = a[1][0]
            #print(f'\t {valor_mano} \t {b}')
            if valor_mano != b:
                return False
        return True
    except:
        return False
    

# Simulación de probabilidades

In [249]:
def calcular_probabilidad( manos ):
    """
    Regresa un conteo de las diferentes manos obtenidas y su porcentaje
    """
    valores_manos = {"pares":0, 
                    "dobles pares":0, 
                    "tercia":0, 
                    "escalera":0,
                    "color" : 0,
                    "full" : 0,
                    "poker" : 0,
                    "escalera de color" : 0, 
                    "escalera_real" : 0,}

    ###
    # Analisis de cada mano
    ###
    for mano in manos:
        valores = []
        palos = []

        if escalera_real(mano):
            valores_mano["escalera_real"] += 1
        for carta in mano:
            valores.append(carta[1])

        counter = dict(collections.Counter(valores))
        list_counter = list(counter.values())
        if list_counter.count(2) == 1:
            valores_manos["pares"] += 1
        elif list_counter.count(2) == 2:
            valores_manos["dobles pares"] += 1
        elif  list_counter.count(3) == 1:
            valores_manos["tercia"] += 1
        if escalera(mano, len(mano)):
            valores_manos["escalera"] += 1
        
        if list_counter.count(4) == 1:
            valores_manos["poker"] += 1

        #Mover posición de está condición
        if list_counter.count(2) == 1 and list(counter.values()).count(3) == 1:
            valores_manos["full"] += 1

        if color(mano, len(mano)):
            valores_manos["color"] += 1    
            if escalera_color(mano, len(mano)):
                valores_manos["escalera de color"] += 1
            
    
    for key, value in valores_manos.items():
        print(f'Probabilidad de obtener {key} es {value}/{len(manos)} = {value/len(manos)* 100}%')

    return valores_manos


In [201]:
def main(tamano_mano, intentos):
    """
    Genera simulación con una n cantidad de intentos
    """
    
    barajas = generar_baraja()
    manos = []

    for _ in range(intentos):
        mano = obtener_mano(barajas, tamano_mano)
        manos.append(mano)


    vals = calcular_probabilidad(manos)



In [252]:
#Introducir variables aquí
tamano_mano = 5
#intentos = 2598960
intentos = 100000
main(tamano_mano, intentos)

Probabilidad de obtener pares es 42450/100000 = 42.449999999999996%
Probabilidad de obtener dobles pares es 4724/100000 = 4.723999999999999%
Probabilidad de obtener tercia es 2181/100000 = 2.181%
Probabilidad de obtener escalera es 396/100000 = 0.396%
Probabilidad de obtener color es 203/100000 = 0.203%
Probabilidad de obtener full es 142/100000 = 0.14200000000000002%
Probabilidad de obtener poker es 20/100000 = 0.02%
Probabilidad de obtener escalera de color es 2/100000 = 0.002%
Probabilidad de obtener escalera_real es 0/100000 = 0.0%


# Tests

---

In [166]:
#Test escalera
mano_1 = generar_baraja()[1:6]
mano_2 = generar_baraja()[2:7]
mano_3 = obtener_mano(generar_baraja(), 5)
mano_4 = generar_baraja()[6:11]
mano_5 = generar_baraja()[7:7+5]

for mano in [mano_1, mano_2, mano_3, mano_4, mano_5]:
    print(mano)
    if escalera(mano, 5):
        print("++++Wuuuuu\n")
    else:
        print("++++Ah:(\n")

[('espada', (2, '2')), ('espada', (3, '3')), ('espada', (4, '4')), ('espada', (5, '5')), ('espada', (6, '6'))]
++++Wuuuuu

[('espada', (3, '3')), ('espada', (4, '4')), ('espada', (5, '5')), ('espada', (6, '6')), ('espada', (7, '7'))]
++++Wuuuuu

[('corazon', (7, '7')), ('trebol', (12, 'Q')), ('trebol', (5, '5')), ('corazon', (1, 'A')), ('corazon', (10, '10'))]
++++Ah:(

[('espada', (7, '7')), ('espada', (8, '8')), ('espada', (9, '9')), ('espada', (10, '10')), ('espada', (11, 'J'))]
++++Wuuuuu

[('espada', (8, '8')), ('espada', (9, '9')), ('espada', (10, '10')), ('espada', (11, 'J')), ('espada', (12, 'Q'))]
++++Wuuuuu



In [73]:
print(mano_2)

[('espada', (3, '3')), ('espada', (4, '4')), ('espada', (5, '5')), ('espada', (6, '6')), ('espada', (7, '7'))]


In [223]:
#Test color
for mano in [mano_1, mano_2]:
    if color(mano, len(mano)):
        print("Yes")
    else:
        print("No")

Yes
Yes
